<a href="https://colab.research.google.com/github/GuillermoJaca/TextGenerationPepaPig/blob/master/PepaPig.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install urlopen

  Created wheel for urlopen: filename=urlopen-1.0.0-cp36-none-any.whl size=1411 sha256=264303959b5194d1d730762f56170c67d80e9bcaeca51963c3eb929a54d8af12
  Stored in directory: /root/.cache/pip/wheels/1e/3e/86/70a25feebbfcbde0fbb2defe9e0610ba44aa32217fc4a53060
Successfully built urlopen


In [ ]:
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import re
import requests

def getPepaText(url):
  #eg. url = 'https://transcripts.fandom.com/wiki/Muddy_Puddles'
  #Get paragraphs files
  r = requests.get(url)
  text = r.text

  soup = BeautifulSoup(text, 'xml')
  yourstring = str(soup.findAll('p'))
  replaced = re.sub("</?p[^>]*>", "", yourstring)
  lines = replaced.splitlines()
  return lines

def get_links(mainhttp):
  #Get all links from web page of transcriptions
  req = Request(mainhttp)
  html_page = urlopen(req)

  soup = BeautifulSoup(html_page, "lxml")

  links = []
  for link in soup.findAll('a'):
      links.append(link.get('href'))

  #Filter web links to strap
  clean_links =[]
  for i in links:
    if i is not None and i.startswith('/wiki'): clean_links.append(i)

  return clean_links


def get_dialoge(names,links,link_to_scrap):

  final_data = []
  for i in links:
    link_to_scrap = 'https://transcripts.fandom.com' + '/' + str(i)
    data = getPepaText(url= link_to_scrap)
    print(link_to_scrap)
    for i in data:
      if isinstance(i,str): 
        aux = i.strip("[")
        aux = aux.strip("]")
        aux = aux.strip(",")
        aux = aux.strip("</i>")
        aux = aux.strip("<i>")
        aux = aux.strip("</b>")
        aux = aux.strip("<b>")
        aux = aux.strip()
        if aux.startswith(tuple(names)): final_data.append(aux)

  return final_data
        

mainhttp = 'https://transcripts.fandom.com/wiki/Category:Peppa_Pig_transcripts' #https://transcripts.fandom.com/wiki/Rebecca_Rabbit
names = ['Peppa', 'George', 'Granny Pig', 'Narrator', 'Grandpa Pig','Daddy Pig', 'Mummy Pig'] #len 5000

links = get_links(mainhttp)
conversations = get_dialoge(names=names,links= links,link_to_scrap=mainhttp)

#Write file
with open('conversations.txt', 'w') as filehandle:
    for listitem in conversations:
        filehandle.write('%s\n' % listitem)

'''
#Load data
conv = []

# open file and read the content in a list
with open('conversations.txt', 'r') as filehandle:
    for line in filehandle:
        # remove linebreak which is the last character of the string
        con = line[:-1]

        # add item to the list
        conv.append(con)
'''

https://transcripts.fandom.com//wiki/Special:CreatePage
https://transcripts.fandom.com//wiki/Cars
https://transcripts.fandom.com//wiki/The_Prince_of_Egypt
https://transcripts.fandom.com//wiki/Antz
https://transcripts.fandom.com//wiki/Con_Job
https://transcripts.fandom.com//wiki/A_Bug%27s_Life
https://transcripts.fandom.com//wiki/Hero_of_the_Rails
https://transcripts.fandom.com//wiki/Dio_the_Invader
https://transcripts.fandom.com//wiki/Category:Complete_Transcripts
https://transcripts.fandom.com//wiki/101_Dalmatians_(1961)
https://transcripts.fandom.com//wiki/A_Night_of_Fright_is_No_Delight
https://transcripts.fandom.com//wiki/A_Clue_for_Scooby-Doo
https://transcripts.fandom.com//wiki/Hassle_in_the_Castle_(Scooby-Doo,_Where_Are_You!)
https://transcripts.fandom.com//wiki/Scooby-Doo!_and_the_Witch%27s_Ghost
https://transcripts.fandom.com//wiki/Spooky_Space_Kook
https://transcripts.fandom.com//wiki/Go_Away_Ghost_Ship
https://transcripts.fandom.com//wiki/Category:Movies
https://transcripts.

"\n#Load data\nconv = []\n\n# open file and read the content in a list\nwith open('conversations.txt', 'r') as filehandle:\n    for line in filehandle:\n        # remove linebreak which is the last character of the string\n        con = line[:-1]\n\n        # add item to the list\n        conv.append(con)\n"

#Load and split data

In [ ]:
#Load data
conv = []

# open file and read the content in a list
with open('conversations.txt', 'r') as filehandle:
    for line in filehandle:
        # remove linebreak which is the last character of the string
        con = line[:-1]

        # add item to the list
        conv.append(con)

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(conversations,test_size=0.15) 

In [ ]:
#Write file
with open('train.txt', 'w') as filehandle:
    for listitem in train:
        filehandle.write('%s\n' % listitem)

In [ ]:
#Write file
with open('test.txt', 'w') as filehandle:
    for listitem in test:
        filehandle.write('%s\n' % listitem)

In [ ]:
!pip install transformers

     |████████████████████████████████| 1.4MB 15.7MB/s 
     |████████████████████████████████| 890kB 51.6MB/s 
     |████████████████████████████████| 2.9MB 48.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=6a0989a1444a048c8ad0f756064e2f675f0dfdf70fb82e3392dd3e2214579374
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2-medium")

train_path = 'train.txt'
test_path = 'test.txt'

In [ ]:
from transformers import TextDataset,DataCollatorForLanguageModeling

def load_dataset(train_path,test_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=20)
     
    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=20)   
    
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset,test_dataset,data_collator

train_dataset,test_dataset,data_collator = load_dataset(train_path,test_path,tokenizer)

/usr/local/lib/python3.6/dist-packages/transformers/data/datasets/language_modeling.py:44: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_mlm.py
  FutureWarning,


In [ ]:
from transformers import Trainer, TrainingArguments,AutoModelWithLMHead

model = AutoModelWithLMHead.from_pretrained("gpt2-medium")


training_args = TrainingArguments(
    output_dir="/content", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=3, # number of training epochs
    per_device_train_batch_size=1, # batch size for training
    per_device_eval_batch_size=1,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved 
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,

)

/usr/local/lib/python3.6/dist-packages/transformers/models/auto/modeling_auto.py:852: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


In [ ]:
trainer.train()
trainer.save_model()

In [ ]:
from transformers import pipeline

Pepastory = pipeline('text-generation',model='/content', tokenizer='gpt2-medium',config={'max_length':800})


In [ ]:
Pepastory('Pepa')